In [1]:
#Importando librerias
import wget
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

#Definiendo las variables globales
global df_traffic, resultados, modelo, modelo_clasificacion

In [2]:
#Extraemos el contenido del DataFrame y aplicamos la función preprocesamiento y tomamos una muestra para verificar las columnas desplegadas
df_traffic = pd.read_csv('https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/traffic_site.csv', dtype={'date':object,'fullVisitorId':object,'visitId':object})
df_traffic.sample(5)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
486,Paid Search,20170630,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",5834024514209517952,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",5834024514209517952_1498832574,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""AW - Dynamic Search Ads Whole Si...",1498832574,1,1498832574
7486,Organic Search,20161025,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7292280031752763212,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",7292280031752763212_1477389474,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1477389474,1,1477389474
2988,Organic Search,20170128,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9723946712961525512,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",9723946712961525512_1485652999,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1485652999,1,1485652999
5276,Social,20161102,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4565505291479424987,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4565505291479424987_1478137233,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""referralPath"": ""/yt/about/"", ""campaign"": ""(n...",1478137233,1,1478137233
1629,Referral,20170412,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4894435742807700896,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",4894435742807700896_1492032897,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""1""}","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1492032897,3,1492032897


In [3]:
#Verificamos el número de columnas y filas contenidas en el DataFrame y su tipo de dato asignado a cada una
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   channelGrouping       12283 non-null  object
 1   date                  12283 non-null  object
 2   device                12283 non-null  object
 3   fullVisitorId         12283 non-null  object
 4   geoNetwork            12283 non-null  object
 5   sessionId             12283 non-null  object
 6   socialEngagementType  12283 non-null  object
 7   totals                12283 non-null  object
 8   trafficSource         12283 non-null  object
 9   visitId               12283 non-null  object
 10  visitNumber           12283 non-null  int64 
 11  visitStartTime        12283 non-null  int64 
dtypes: int64(2), object(10)
memory usage: 1.1+ MB


In [4]:
# Verificamos el tipo de dato contenido en las columnas anidadas
df_traffic.device.value_counts().index

Index(['{"browser": "Chrome", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "operatingSystem": "Windows", "operatingSystemVersion": "not available in demo dataset", "isMobile": false, "mobileDeviceBranding": "not available in demo dataset", "mobileDeviceModel": "not available in demo dataset", "mobileInputSelector": "not available in demo dataset", "mobileDeviceInfo": "not available in demo dataset", "mobileDeviceMarketingName": "not available in demo dataset", "flashVersion": "not available in demo dataset", "language": "not available in demo dataset", "screenColors": "not available in demo dataset", "screenResolution": "not available in demo dataset", "deviceCategory": "desktop"}',
       '{"browser": "Chrome", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "operatingSystem": "Macintosh", "operatingSystemVersion": "not available in demo dataset", "isMobile": false, "mobileDeviceBr

In [5]:
def preprocesamiento():
    global df_traffic

    # Extraer datos de diccionarios
    diccionarios = ['device','geoNetwork','trafficSource','totals']
    for columna in diccionarios:
        df_traffic = df_traffic.join(pd.DataFrame([json.loads(linea) for linea in df_traffic[columna]]))
    df_traffic = df_traffic.drop(columns=diccionarios)
    # o puedo ustilizar esta otra opción df_traffic.drop(columns=diccionarios, inplace=True)

    #Eliminar las columnas que no requerimos
    df_traffic= df_traffic.drop(columns=['socialEngagementType','browserVersion','browserSize','operatingSystemVersion','mobileDeviceBranding','mobileDeviceModel','mobileInputSelector','mobileDeviceInfo','mobileDeviceMarketingName','flashVersion','language','screenColors','screenResolution','continent','subContinent','country','region','cityId','latitude','longitude','networkLocation','keyword','adwordsClickInfo','isTrueDirect','adContent','campaignCode','networkDomain','visits'])

    # Eliminar duplicados
    df_traffic = df_traffic.drop_duplicates()

    # Reemplazar textos por números
    cuant = ['bounces', 'hits','newVisits','pageviews', 'transactionRevenue']
    for columna in cuant:
        df_traffic[columna] = pd.to_numeric(df_traffic[columna])
    df_traffic[cuant] = df_traffic[cuant].fillna(0)
    df_traffic['transactionRevenue'] = df_traffic['transactionRevenue'] / 1000000


   # Reemplazar números por fechas
    df_traffic['date'] = pd.to_datetime(df_traffic['date'], format='%Y%m%d')
    df_traffic['visitStartTime'] = pd.to_datetime(df_traffic['visitStartTime'], unit='s')
    df_traffic.bounces = df_traffic.bounces.astype(int)
    df_traffic.newVisits = df_traffic.newVisits.astype(int)



preprocesamiento()
df_traffic.sample(5)

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,browser,operatingSystem,isMobile,deviceCategory,metro,city,campaign,source,medium,referralPath,hits,pageviews,bounces,newVisits,transactionRevenue
2741,Organic Search,2016-09-15,7557198742018733762,7557198742018733762_1474005676,1474005676,1,2016-09-16 06:01:16,Chrome,Android,True,mobile,not available in demo dataset,not available in demo dataset,(not set),google,organic,NaN,8,5,0,1,0.0
2293,Organic Search,2017-02-17,1888716018748123270,1888716018748123270_1487394633,1487394633,2,2017-02-18 05:10:33,Chrome,Linux,False,desktop,Los Angeles CA,Irvine,(not set),google,organic,NaN,1,1,1,0,0.0
9927,Social,2016-11-05,380808794002813809,0380808794002813809_1478379345,1478379345,1,2016-11-05 20:55:45,Chrome,Windows,False,desktop,not available in demo dataset,not available in demo dataset,(not set),youtube.com,referral,/yt/about/es/,1,1,1,1,0.0
4288,Direct,2017-03-31,4287946192104966450,4287946192104966450_1490954782,1490954782,1,2017-03-31 10:06:22,Firefox,Macintosh,False,desktop,not available in demo dataset,not available in demo dataset,(not set),(direct),(none),NaN,2,2,0,1,0.0
5056,Organic Search,2017-04-05,4668034960473162715,4668034960473162715_1491431771,1491431771,1,2017-04-05 22:36:11,Chrome,Android,True,tablet,not available in demo dataset,not available in demo dataset,(not set),google,organic,NaN,1,1,1,1,0.0


In [6]:
#Verificamos el número de columnas y filas contenidas en el DataFrame y su tipo de dato asignado a cada una
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   channelGrouping     12283 non-null  object        
 1   date                12283 non-null  datetime64[ns]
 2   fullVisitorId       12283 non-null  object        
 3   sessionId           12283 non-null  object        
 4   visitId             12283 non-null  object        
 5   visitNumber         12283 non-null  int64         
 6   visitStartTime      12283 non-null  datetime64[ns]
 7   browser             12283 non-null  object        
 8   operatingSystem     12283 non-null  object        
 9   isMobile            12283 non-null  bool          
 10  deviceCategory      12283 non-null  object        
 11  metro               12283 non-null  object        
 12  city                12283 non-null  object        
 13  campaign            12283 non-null  object    